In [36]:
from langchain.document_loaders import DirectoryLoader,TextLoader,PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from langchain.chains import RetrievalQA
import gradio as gr
import ollama 
import os
import glob
import shutil

In [37]:
!pip install -U langchain-ollama

In [38]:
!pip install pymupdf

In [40]:
!pip install -U langchain-ollama

In [54]:
! pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to c:\users\surya\appdata\local\temp\pip-req-build-9tkyt4ao
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\surya\AppData\Local\Temp\pip-req-build-9tkyt4ao'


In [ ]:
import whisper
def speech_text(audio):
    if audio:
        print(audio)
    model=whisper.load_model("base")
    result=model.transcribe(audio)
    return result["text"]


In [39]:
db_name="vector_db"

In [51]:
import gradio as gr
import os
import shutil
from langchain.document_loaders import PyMuPDFLoader
from langchain_ollama import ChatOllama

conversation_chain=None
def process_pdf(pdf_file):
    global conversation_chain
    UPLOAD_FOLDER = "./data"
    if os.path.exists(UPLOAD_FOLDER):
        for filename in os.listdir(UPLOAD_FOLDER):
            filepath=os.path.join(UPLOAD_FOLDER,filename)
            if os.path.isfile(filepath) or os.path.islink(filepath):
                os.remove(filepath)
    else:
        os.mkdir(UPLOAD_FOLDER)

    
    shutil.copy(pdf_file.name, UPLOAD_FOLDER)
    saved_path = os.path.join(UPLOAD_FOLDER, os.path.basename(pdf_file.name))
    # print(saved_path,os.path.basename(pdf_file.name))
    loader = PyMuPDFLoader(saved_path)
    documents = loader.load()  
    text_splitter = CharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_documents(documents)
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    if os.path.exists(db_name):
        Chroma(persist_directory=db_name,embedding_function=embeddings).delete_collection()
    vectorstore=Chroma.from_documents(documents=chunks,embedding=embeddings,persist_directory=db_name)
    llm=ChatOllama(model="llama3.2",temperature=0.7)
    memory=ConversationBufferMemory(memory_key="chat_history",return_messages=True)
    retriever=vectorstore.as_retriever()
    conversation_chain=ConversationalRetrievalChain.from_llm(llm=llm,retriever=retriever,memory=memory)
    return "✅ PDF processed and QA system ready! You can now ask questions."

    




In [77]:
def chat(message,history):
    global conversation_chain
    result=conversation_chain.invoke({"question":message})
    return result["answer"]
with gr.Blocks() as demo:
    gr.Markdown("## 📄 Upload PDF & Ask Questions")

    pdf_input = gr.File(label="Upload PDF", file_types=[".pdf"])
    upload_status = gr.Textbox(label="Status", interactive=False)
    gr.Markdown("## You can either speek or type your question")
    question_audio=gr.Audio(sources="microphone",type="filepath")
    transcribe=gr.Button("Transcribe")
    question_input = gr.Textbox(label="Your Question")
    transcribe.click(fn=speech_text,inputs=question_audio,outputs=question_input)
    answer_output = gr.Textbox(label="Answer", interactive=False, lines=5)

    ask_button = gr.Button("Ask")

    pdf_input.change(fn=process_pdf, inputs=pdf_input, outputs=upload_status)
    ask_button.click(fn=chat, inputs=question_input, outputs=answer_output)

demo.launch(inbrowser=True,share=True)


c:\Users\surya\anaconda3\envs\llms\Lib\site-packages\gradio\utils.py:1023: UserWarning: Expected 2 arguments for function <function chat at 0x00000168141F3F60>, received 1.
  warnings.warn(
c:\Users\surya\anaconda3\envs\llms\Lib\site-packages\gradio\utils.py:1027: UserWarning: Expected at least 2 arguments for function <function chat at 0x00000168141F3F60>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7897
* Running on public URL: https://4c6adf3a4d6c368e43.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
